## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from equilibrator_api import Q_, R, default_T
import model_balancing

In [2]:
args = {}

# Network topology

args["S"] = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)
args["A_act"] = np.matrix(
    [
        [0, 1, 2],
        [0, 0, 0],
        [0, 0, 0]
    ], dtype=float
)
args["A_inh"] = np.matrix(
    [
        [0, 0, 0],
        [0, 0, 0],
        [2, 0, 0]
    ], dtype=float
)

In [3]:
# Priors for Kinetic and Thermodynamic parameters

args["Keq_gmean"] = np.array([57, 0.13, 3200])  # geometric mean (assuming a standard concentration of 1M)
args["Keq_ln_cov"] = np.array(
    [
        [4, 0, 0],
        [0, 4, 0],
        [0, 0, 4],
    ]
)  # log-scale covariance

args["kcatf_gmean"] = Q_([20.0, 8.0, 10.0], "1/s")  # geometric mean
args["kcatf_ln_cov"] = np.array(
    [
        [0.48, 0, 0],
        [0, 5.3, 0],
        [0, 0, 5.3],
    ]
) # log-scale covariance

args["kcatr_gmean"] = Q_([3.5e-3, 6.2e4, 3.1e-1], "1/s")  # geometric mean
args["kcatr_ln_cov"] = np.array(
    [
        [21.2, 0, 0],
        [0, 21.2, 0],
        [0, 0, 21.2],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
args["Km_gmean"] = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
args["Km_ln_cov"] = np.array(
    [
        [5.3, 0, 0, 0, 0, 0],
        [0, 5.3, 0, 0, 0, 0],
        [0, 0, 5.3, 0, 0, 0],
        [0, 0, 0, 5.3, 0, 0],
        [0, 0, 0, 0, 5.3, 0],
        [0, 0, 0, 0, 0, 5.3],
    ]
) # log-scale covariance

args["Ka_gmean"] = Q_([1, 1], "M")
args["Ka_ln_cov"] = np.array(
    [
        [21.2, 0],
        [0, 21.2],
    ]
)

args["Ki_gmean"] = Q_([1], "M")
args["Ki_ln_cov"] = np.array(
    [
        [21.2],
    ]
)

In [33]:
# condition dependent data (columns represent conditions)

args["fluxes"] = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

args["conc_enz_gmean"] = Q_(
    [
#        [1e-3, 1e-3, 1e-3, 3e-3],
#        [2e-3, 3e-3, 1e-3, 2e-3],
#        [1e-3, 2e-3, 2e-3, 1e-3],
        [1.00177750e-04, 7.50146605e-05, 1.25332322e-04, 1.25222188e-04],
        [1.01765625e-01, 5.47083333e-02, 1.68439655e-01, 2.71104167e-01],
        [2.51004922e+01, 5.10065940e-01, 1.25501807e+01, 2.00406463e+02],
    ], "M")  # geometric mean

args["conc_enz_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

args["conc_met_gmean"] = Q_(
    [
        [2e-3, 1e-2, 2e-3, 1e-3],
        [2e-4, 1e-4, 3e-4, 1e-4],
        [1e-5, 1e-5, 1e-5, 1e-5],
    ], "M"
)  # geometric mean

args["conc_met_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

In [34]:
fluxes, S, A_act, A_inh = args["fluxes"], args["S"], args["A_act"], args["A_inh"]
ln_Keq = np.log(args["Keq_gmean"])
ln_kcatf = np.log(args["kcatf_gmean"].m_as("1/s"))
ln_Km = np.log(args["Km_gmean"].m_as("M"))
ln_Ka = np.log(args["Ka_gmean"].m_as("M"))
ln_Ki = np.log(args["Ki_gmean"].m_as("M"))
ln_conc_met = np.log(args["conc_met_gmean"].m_as("M"))

print("enzyme concentrations: \n", 
    np.exp(
        model_balancing._ln_conc_enz(
            ln_Keq, ln_kcatf, ln_Km, ln_Ka, ln_Ki, ln_conc_met,
            fluxes, S, A_act, A_inh , rate_law="S").value
    )
)

print(
    "kcatr: \n", 
    np.exp(model_balancing._ln_kcatr(ln_kcatf, ln_Km, ln_Keq, S).value)
)

enzyme concentrations: 
 [[1.00177750e-04 7.50146605e-05 1.25332322e-04 1.25222188e-04]
 [1.01765625e-01 5.47083333e-02 1.68439655e-01 2.71104167e-01]
 [2.51004922e+01 5.10065940e-01 1.25501807e+01 2.00406463e+02]]
kcatr: 
 [3.50877193e-03 6.15384615e+04 3.12500000e-01]


In [35]:
for rate_law in ["S"]:#["S", "1S", "SP", "1SP", "CM"]:
    print(f"************ rate law = {rate_law} **************\n")
    model_balancing.solve(
        **args,
        rate_law=rate_law
    )

************ rate law = S **************


Metabolite concentrations (M) =
 [[2.01147644e-03 9.99996310e-03 2.00005054e-03 9.99993010e-04]
 [2.00012315e-04 1.00005903e-04 3.00008343e-04 1.00005908e-04]
 [9.99928542e-06 9.99927648e-06 9.99925604e-06 9.99926613e-06]]

Enzyme concentrations (M) =
 [[5.14538167e-05 3.84553019e-05 6.44452397e-05 6.43187132e-05]
 [9.01512152e-04 6.84143032e-04 1.69675487e-03 8.38668335e-04]
 [1.25882056e-04 3.73680494e-05 6.31473265e-05 4.19343572e-04]]

Driving forces (RT) =
 [[ 5.54942682  7.84628896  5.13829893  5.54370051]
 [ 1.8051896   1.11204076  2.21062388  1.11204185]
 [10.72823228 10.03508344 11.13366656 10.03508453]]

η(thr) =
 [[1.   1.   0.99 1.  ]
 [0.84 0.67 0.89 0.67]
 [1.   1.   1.   1.  ]]

η(kin) =
 [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]

η(reg) =
 [[1.   1.   1.   1.  ]
 [0.51 0.84 0.51 0.34]
 [0.51 0.86 0.51 0.3 ]]




All Z-scores
--------------------------------------------------
enzymes =  0.0
metabolites =  0.0
Keq =  0.37
kca

In [9]:
# enzymes =  3.43
# metabolites =  3.26
# Keq =  2.43
# kcat forward =  1.95
# kcat reverse =  1.89
# Km =  2.04
# Ka =  4.31
# Ki =  0.0